In [5]:
from newsapi import NewsApiClient
import os

In [6]:
NEWS_KEY = os.getenv('NEWS_KEY')

In [7]:
newsapi = NewsApiClient(api_key= NEWS_KEY)

In [9]:
top_headlines = newsapi.get_top_headlines(q='bitcoin',
#                                           sources='bbc-news,the-verge',
                                          category='business',
                                          language='en',
                                          country='us')

In [10]:
top_headlines

{'status': 'ok',
 'totalResults': 1,
 'articles': [{'source': {'id': None, 'name': 'Newsbtc.com'},
   'author': None,
   'title': 'Bitfinex Whale Who Warned About Bitcoin Fiat Inflow Is Up $2M in Profit - newsBTC',
   'description': "One Bitcoin whale recently noted that fiat in/outflows are key to determining BTC's trend, and he appears to have earned nearly $2 million in profits since.",
   'url': 'https://www.newsbtc.com/2020/02/04/bitfinex-whale-who-warned-about-bitcoin-fiat-inflow-is-up-2m-in-profit/',
   'urlToImage': 'https://www.newsbtc.com/wp-content/uploads/2020/02/shutterstock_721420015-1200x780.jpg',
   'publishedAt': '2020-02-04T23:03:45Z',
   'content': 'Bitcoin has seen some turbulent price action over the past couple of days, with the cryptocurrency briefly rallying to highs of over $9,600 before incurring a swift selloff that has sent BTC down into the lower-$9,000 region.\r\nThe drop that has occurred over … [+2707 chars]'}]}

In [16]:
f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=oil,naturalgas&api-key=Bmf4GKCtrsMvIyxf7DFl8HefYqv3cZK5'

'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=oil,naturalgas&api-key=Bmf4GKCtrsMvIyxf7DFl8HefYqv3cZK5'

In [39]:
import requests
requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?q=oil,naturalgas&api-key=Bmf4GKCtrsMvIyxf7DFl8HefYqv3cZK5').json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2020 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'abstract': 'Texaco expects to supply its home-heating oil customers this winter with at least 87% of amount of fuel they received a year ago (S)',
    'web_url': 'https://www.nytimes.com/1979/09/08/archives/texaco-fueloil-supply-at-87.html',
    'snippet': 'Texaco expects to supply its home-heating oil customers this winter with at least 87% of amount of fuel they received a year ago (S)',
    'lead_paragraph': 'Texaco Inc. said yesterday that it expected to supply its home‐heating oil customers this winter with at least 87 percent of the amount of fuel they received a year ago.',
    'print_page': '31',
    'source': 'The New York Times',
    'multimedia': [],
    'headline': {'main': 'Texaco Fuel‐Oil Supply at 87%',
     'kicker': None,
     'content_kicker': None,
     'print_headline': 'Texaco Fuel‐Oil Supply at 87%',
     'name': None,
     'seo': None,
  

In [28]:
!pip install requests